In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model
import deeplake
import numpy as np
from sklearn.model_selection import train_test_split


2023-09-18 19:35:29.791325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 19:35:30.972888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Загрузка данных
ds = deeplake.load('hub://activeloop/not-mnist-small')

-

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/not-mnist-small



-

hub://activeloop/not-mnist-small loaded successfully.



In [3]:
ds

Dataset(path='hub://activeloop/not-mnist-small', read_only=True, tensors=['images', 'labels'])

In [4]:
# Преобразование данных в numpy массивы
images = np.array(ds['images'])
labels = np.array(ds['labels'])

# Разбиение данных на тренировочный и тестовый наборы
notmnist_train, notmnist_test, _, _ = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
# Загрузка и предобработка данных
(fashion_train, fashion_train_labels), (fashion_test, fashion_test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Нормализация данных
fashion_train = fashion_train.astype('float32') / 255.
fashion_test = fashion_test.astype('float32') / 255.

# Архитектура автоэнкодера
input_img = Input(shape=(28, 28))
x = Flatten()(input_img)
x = Dense(128, activation='relu')(x)
encoded = Dense(64, activation='relu')(x)

x = Dense(128, activation='relu')(encoded)
x = Dense(28*28, activation='sigmoid')(x)
decoded = Reshape((28, 28))(x)

autoencoder = Model(input_img, decoded)

# Обучение автоэнкодера на notMNIST
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(notmnist_train, notmnist_train, epochs=50, batch_size=256, validation_data=(notmnist_test, notmnist_test))

# Создание модели для Fashion MNIST на основе весов кодировщика
encoder = Model(input_img, encoded)
for layer in encoder.layers:
    layer.trainable = False

x = Dense(128, activation='relu')(encoded)
predictions = Dense(10, activation='softmax')(x)

fashion_model = Model(input_img, predictions)
fashion_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
fashion_model.fit(fashion_train, fashion_train_labels, epochs=50, batch_size=256, validation_data=(fashion_test, fashion_test_labels))

# Предсказание
predictions = fashion_model.predict(fashion_test)


2023-09-18 19:35:43.549837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-18 19:35:43.669421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-18 19:35:43.669830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-18 19:35:43.672594: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-18 19:35:43.673163: I tensorflow/compile

Epoch 1/50


2023-09-18 19:35:47.605114: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-18 19:35:47.635470: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f87fc04d580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-18 19:35:47.635541: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2023-09-18 19:35:47.668544: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-18 19:35:47.906966: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-18 19:35:48.062216: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-18 19:

59/59 [==============================] - 4s 8ms/step - loss: -1375.5360 - val_loss: -1539.6726
Epoch 2/50
59/59 [==============================] - 0s 5ms/step - loss: -1544.6702 - val_loss: -1550.4723
Epoch 3/50
59/59 [==============================] - 0s 6ms/step - loss: -1552.4774 - val_loss: -1556.3253
Epoch 4/50
59/59 [==============================] - 0s 6ms/step - loss: -1554.0099 - val_loss: -1556.3252
Epoch 5/50
59/59 [==============================] - 0s 5ms/step - loss: -1554.0100 - val_loss: -1556.3253
Epoch 6/50
59/59 [==============================] - 0s 5ms/step - loss: -1554.0100 - val_loss: -1556.3253
Epoch 7/50
59/59 [==============================] - 0s 5ms/step - loss: -1554.0101 - val_loss: -1556.3253
Epoch 8/50
59/59 [==============================] - 0s 5ms/step - loss: -1554.0104 - val_loss: -1556.3253
Epoch 9/50
59/59 [==============================] - 0s 5ms/step - loss: -1554.0099 - val_loss: -1556.3256
Epoch 10/50
59/59 [==============================] - 0s 5

235/235 [==============================] - 1s 4ms/step - loss: 0.7619 - accuracy: 0.7192 - val_loss: 0.7685 - val_accuracy: 0.7220
Epoch 22/50
235/235 [==============================] - 1s 4ms/step - loss: 0.7555 - accuracy: 0.7205 - val_loss: 0.7645 - val_accuracy: 0.7201
Epoch 23/50
235/235 [==============================] - 1s 5ms/step - loss: 0.7522 - accuracy: 0.7216 - val_loss: 0.7598 - val_accuracy: 0.7270
Epoch 24/50
235/235 [==============================] - 1s 5ms/step - loss: 0.7457 - accuracy: 0.7258 - val_loss: 0.7640 - val_accuracy: 0.7202
Epoch 25/50
235/235 [==============================] - 1s 4ms/step - loss: 0.7447 - accuracy: 0.7263 - val_loss: 0.7596 - val_accuracy: 0.7235
Epoch 26/50
235/235 [==============================] - 1s 4ms/step - loss: 0.7405 - accuracy: 0.7263 - val_loss: 0.7820 - val_accuracy: 0.7014
Epoch 27/50
235/235 [==============================] - 1s 4ms/step - loss: 0.7379 - accuracy: 0.7265 - val_loss: 0.7569 - val_accuracy: 0.7262
Epoch 28/50

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Измените размер изображений и преобразуйте в трехканальный формат
train_images = np.repeat(np.resize(train_images, (train_images.shape[0], 224, 224)), 3, axis=-1).reshape(train_images.shape[0], 224, 224, 3)
test_images = np.repeat(np.resize(test_images, (test_images.shape[0], 224, 224)), 3, axis=-1).reshape(test_images.shape[0], 224, 224, 3)

# Нормализуйте изображения
train_images = train_images.astype('float32') / 255.
test_images = test_images.astype('float32') / 255.


In [ ]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Добавьте новые верхние слои
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Заморозьте слои базовой модели VGG16
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=5, validation_data=(test_images, test_labels))


In [ ]:
# Разморозка верхних слоев
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=5, validation_data=(test_images, test_labels))
